Assuming you have already extracted the files

In [1]:
import re
#number of entries in this zip: 996181
filename= "s2-corpus-000"
filedir= "/home/ricostynha/Desktop/Erasmus/Apps/FoolingAround"
filepath = filedir+"/"+filename

In [2]:
#just for modularity read the first 10000 entries and save in a file just to have one
Maxentries = 10000;
dstname = "fist10000papers.txt"
dstpath = filedir+"/"+dstname;
file_list = []
dstfile= open(dstpath,"w+")
with open(filepath) as infile:
    for line in infile:
        Maxentries-=1
        file_list.append(line)
        dstfile.write(line);
        if(Maxentries==0):
            break
dstfile.close()

In [3]:
def parsingraw_entry(raw):
    #year is ommisee for web pages , so it will have less fields
    fields = ['"entities":','"journalVolume":','"journalPages":','"pmid":','"year":','"outCitations":','"s2Url":','"s2PdfUrl"',
         '"id":','"authors":','"journalName":','"paperAbstract":','"inCitations":','"pdfUrls":',
         '"title":','"doi":','"sources":','"doiUrl":','"venue":']
    helper_list=[]
    for field in fields:
        match = re.search(field,raw)
        if(match != None):
            helper_list.append([match.group(),match.span()])

    reference_list=[]
    for i in range(0,len(helper_list)-1):
        begin = helper_list[i][1][1]
        end   = helper_list[i+1][1][0]
        reference_list.append([helper_list[i][0],raw[begin:end]])
    begin = helper_list[-1][1][1]
    reference_list.append([helper_list[-1][0],raw[begin:]])
    return reference_list
   
    

In [4]:
data = []
for i in range(10000):
    a = parsingraw_entry(file_list[i])
    if(len(a)==19):
        c = [b[1] for b in a]
        data.append(c)
    else:
        c1 = [b[1] for b in a[0:4]]
        c2 = [None]
        c3 = [b[1] for b in a[4:]]
        data.append(c1+c2+c3)
  
        
        

In [5]:
import pandas as pd
titles  = ['entities','journalVolume','journalPages','pmid','year:','outCitations','s2Url','s2PdfUrl',
'id','authors','journalName','paperAbstract','inCitations','pdfUrls',
'title','doi','sources','doiUrl','venue']

df = pd.DataFrame(data,columns=titles)

In [6]:
#replacing empty things by none
df = df.replace({"[],":None})
df = df.replace({'"",':None})
df = df.replace({':""':None})
df = df.replace({'""}\n':None})
df = df.replace({':"",':None})

In [9]:
df.to_csv("first1000entries")
